In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's Embedding Models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#models) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt GeminiTextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id in gcloud
#! gcloud config set project {PROJECT_ID}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [3]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [4]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [5]:
import bigframes.pandas as bf

# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bf.options.bigquery.project = PROJECT_ID

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [6]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [7]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.peek(n=5) # View an arbitrary five complaints

,consumer_complaint_narrative
3274540,This is my XXXX request that I have been a vic...
3274638,This complaint is in regards to the Student Lo...
3275041,XXXX XXXX XXXX keeps raising my monthly paymen...
3275069,"A check was stolen from my purse, forged, and ..."
3275144,In accordance with the Fair Credit Reporting a...


Downsample DataFrame to 10,000 records for model training.

In [8]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [9]:
from bigframes.ml.llm import TextEmbeddingGenerator

model = TextEmbeddingGenerator() # No connection id needed

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/core/log_adapter.py:164: FutureWarning: Since upgrading the default model can cause unintended breakages, the
default model will be removed in BigFrames 3.0. Please supply an
explicit model to avoid this message.
  return method(self, *args, **kwargs)


In [10]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/core/array_value.py:109: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
170,[ 1.88376904e-02 -5.30475788e-02 2.95045757e-...,"{""token_count"":187,""truncated"":false}",,I recently had a credit score of XXXX. In XXXX...
1054,[ 1.62359420e-02 4.00104094e-03 3.32496352e-...,"{""token_count"":156,""truncated"":false}",,Hello. I have previously submitted a complaint...
1074,[ 4.31365222e-02 -6.14477554e-03 5.48561616e-...,"{""token_count"":353,""truncated"":false}",,I am disputing an account balance of {$200.00}...
1568,[ 3.44086066e-02 -5.08038402e-02 -2.13879813e-...,"{""token_count"":155,""truncated"":false}",,Someone attempted to open a XXXX XXXX Credit C...
1593,[ 0.005157 -0.03788725 -0.00763594 0.023023...,"{""token_count"":382,""truncated"":false}",,My name is XXXX XXXX. I have been a customer s...
1823,[ 4.99299690e-02 -4.30836529e-02 1.92499068e-...,"{""token_count"":151,""truncated"":false}",,From XX/XX/XXXX to XX/XX/XXXX my Transunion cr...
2004,[ 1.48278452e-03 2.14069756e-03 3.08651775e-...,"{""token_count"":159,""truncated"":false}",,I have written five dispute letters to each re...
2666,[ 1.67662315e-02 -2.69244146e-02 -1.35911610e-...,"{""token_count"":185,""truncated"":false}",,I participated in XXXX XXXX XXXX COVID 19 mort...
3757,[ 1.00584673e-02 -4.75329943e-02 4.52397019e-...,"{""token_count"":555,""truncated"":false}",,I opened a business bank account for my compan...
3854,[ 1.21343853e-02 -2.70731524e-02 -4.29241825e-...,"{""token_count"":28,""truncated"":false}",,XXXX IS STILL ALLOWING FRAUDULENT INFORMATION ...


The model may have encountered errors while calculating embeddings for some rows. Filter out the errored rows before training the model. Alternatively, select these rows and retry the embeddings.

In [11]:
successful_rows = (
    (predicted_embeddings["ml_generate_embedding_status"] == "")
    # Series.str.len() gives the length of an array.
    # See: https://stackoverflow.com/a/41340543/101923
    & (predicted_embeddings["ml_generate_embedding_result"].str.len() != 0)
)
predicted_embeddings = predicted_embeddings[successful_rows]
predicted_embeddings


,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
170,[ 1.88376904e-02 -5.30475788e-02 2.95045757e-...,"{""token_count"":187,""truncated"":false}",,I recently had a credit score of XXXX. In XXXX...
1054,[ 1.62359420e-02 4.00104094e-03 3.32496352e-...,"{""token_count"":156,""truncated"":false}",,Hello. I have previously submitted a complaint...
1074,[ 4.31365222e-02 -6.14477554e-03 5.48561616e-...,"{""token_count"":353,""truncated"":false}",,I am disputing an account balance of {$200.00}...
1568,[ 3.44086066e-02 -5.08038402e-02 -2.13879813e-...,"{""token_count"":155,""truncated"":false}",,Someone attempted to open a XXXX XXXX Credit C...
1593,[ 0.005157 -0.03788725 -0.00763594 0.023023...,"{""token_count"":382,""truncated"":false}",,My name is XXXX XXXX. I have been a customer s...
1823,[ 4.99299690e-02 -4.30836529e-02 1.92499068e-...,"{""token_count"":151,""truncated"":false}",,From XX/XX/XXXX to XX/XX/XXXX my Transunion cr...
2004,[ 1.48278452e-03 2.14069756e-03 3.08651775e-...,"{""token_count"":159,""truncated"":false}",,I have written five dispute letters to each re...
2666,[ 1.67662315e-02 -2.69244146e-02 -1.35911610e-...,"{""token_count"":185,""truncated"":false}",,I participated in XXXX XXXX XXXX COVID 19 mort...
3757,[ 1.00584673e-02 -4.75329943e-02 4.52397019e-...,"{""token_count"":555,""truncated"":false}",,I opened a business bank account for my compan...
3854,[ 1.21343853e-02 -2.70731524e-02 -4.29241825e-...,"{""token_count"":28,""truncated"":false}",,XXXX IS STILL ALLOWING FRAUDULENT INFORMATION ...


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [12]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [13]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["ml_generate_embedding_result"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.peek(n=5)

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/core/array_value.py:109: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,content
371364,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.806875556526...",[ 1.42711382e-02 -7.13259429e-02 4.15564841e-...,"{""token_count"":6,""truncated"":false}",,Please remove all inquiries and collections
368381,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.843635669356...",[ 2.80436985e-02 -5.09077348e-02 -3.63913202e-...,"{""token_count"":6,""truncated"":false}",,clearing out of my debt.
2951416,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.927767679789...",[ 4.07935455e-02 -2.89720390e-03 -6.77521434e-...,"{""token_count"":7,""truncated"":false}",,constant calling to workplace and threats.
2167760,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.755490984751...",[ 3.76710109e-02 -8.52223893e-04 -2.53430642e-...,"{""token_count"":8,""truncated"":false}",,This debt has been paid in full.
1269756,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.770576934220...",[ 5.26239222e-04 -1.80254206e-02 -2.83008888e-...,"{""token_count"":8,""truncated"":false}",,I have no knowledge of said debt.


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use Gemini to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt GeminiTextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [14]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [15]:
# Build plain-text prompts to send to Gemini. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. I have XXXX XXXX XXXX but the hospital and the debt collector conspired to collect this debt and damage my credit. They both never attempted to verify the debt or get paid from the insurance company. They must contact the insurance company before attempting to collect this debt. Also, I never used hospital services in XXXX How did the hospital come up with this debt? I was afraid of going to the hospital due to pandemic in XXXX. How did I get charged for services not received?
2. This letter is being sent to you in response to your recent letter dated XX/XX/XXXX ; which wasnt received at my home until XX/XX/XXXX. This is not a refusal to pay, but a notice that your claim is disputed. Please note this is my second letter to you and I have now sent this certified mail and I will know when you receive it. 

In my first letter, I stated that I applied for financial Aid through the provider. The same day that was denied, my account was sent to collections without giving m

In [16]:
# The plain English request we will make of Gemini
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. I have XXXX XXXX XXXX but the hospital and the debt collector conspired to collect this debt and damage my credit. They both never attempted to verify the debt or get paid from the insurance company. They must contact the insurance company before attempting to collect this debt. Also, I never used hospital services in XXXX How did the hospital come up with this debt? I was afraid of going to the hospital due to pandemic in XXXX. How did I get charged for services not received?
2. This letter is being sent to you in response to your recent letter dated XX/XX/XXXX ; which wasnt received at my home until XX/XX/XXXX. This is not a refusal to pay, but a notice that your claim is disputed. Please note this is my second letter to you and I have now sent this certified mail and I will know when you receive it. 

In my first letter, I stated that I applied for financial Aid through the provider. T

Get a response from Gemini by making a call to Vertex AI using our connection.

In [17]:
from bigframes.ml.llm import GeminiTextGenerator

q_a_model = GeminiTextGenerator(model_name="gemini-1.5-flash-002")

In [18]:
# Make a DataFrame containing only a single row with our prompt for Gemini
df = bf.DataFrame({"prompt": [prompt]})

In [19]:
# Send the request for Gemini to generate a response to our prompt
major_difference = q_a_model.predict(df)
# Gemini's response is the only row in the dataframe result
major_difference["ml_generate_text_llm_result"].iloc[0]

/usr/local/google/home/arwas/src1/python-bigquery-dataframes/bigframes/core/array_value.py:109: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


'The most obvious difference is the **focus of the complaints**.\n\nComment List 1 focuses on issues with **debt collection practices**, including alleged violations of the Fair Debt Collection Practices Act (FDCPA), HIPAA violations, and generally aggressive or unfair collection tactics.  The complaints center around disputes over the validity of debts, improper collection methods, and the emotional distress caused by collectors.\n\nComment List 2 focuses on **identity theft and fraudulent accounts**. The complaints primarily concern inaccurate or fraudulent information reported to credit bureaus, violations of the Fair Credit Reporting Act (FCRA), and the failure of credit reporting agencies and data furnishers to adequately address the reported identity theft.  While some complaints mention debt, the core issue is the fraudulent opening of accounts and the subsequent damage to credit reports.\n'

We now see GeminiTextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).